## Deploy Model as Azure Machine Learning Web Service using MLflow

This example shows you how to use mlflow together with Azure Machine Learning services for deploying a model as a web service. You'll learn how to:

 1. Retrieve a previously trained scikit-learn model
 2. Create a Docker image from the model
 3. Deploy the model as a web service on Azure Container Instance
 4. Make a scoring request against the web service.

## Prerequisites and Set-up

This notebook requires you to first complete the [Use MLflow with Azure Machine Learning for Local Training Run](../train-local/train-local.ipnyb) or [Use MLflow with Azure Machine Learning for Remote Training Run](../train-remote/train-remote.ipnyb) notebook, so as to have an experiment run with uploaded model in your Azure Machine Learning Workspace.

Also install following packages if you haven't already

```
pip install azureml-mlflow pandas
```

Then, import necessary packages:

<img src="https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true">

<img src="https://mlflow.org/docs/0.2.1/_static/MLflow-logo-final-black.png" width="400">

In [1]:
import mlflow
import azureml.mlflow
import azureml.core
from azureml.core import Workspace

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.72


### Connect to workspace and set MLflow tracking URI

Setting the tracking URI is required for retrieving the model and creating an image using the MLflow APIs.

In [2]:
ws = Workspace.from_config()

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

### Retrieve model from previous run

Let's retrieve the experiment from training notebook, and list the runs within that experiment.

In [3]:
experiment_name = "experiment-with-mlflow"
exp = ws.experiments[experiment_name]

runs = list(exp.get_runs())
runs

[Run(Experiment: experiment-with-mlflow,
 Id: 0f226eed-f0e4-4159-9252-e97b2d8bc668,
 Type: None,
 Status: Completed), Run(Experiment: experiment-with-mlflow,
 Id: 6661efb1-e304-4e65-9f75-977248556f17,
 Type: None,
 Status: Completed), Run(Experiment: experiment-with-mlflow,
 Id: e80a4489-0d93-4c6c-95ea-4e83ba5a8488,
 Type: None,
 Status: Completed), Run(Experiment: experiment-with-mlflow,
 Id: 56c4211f-460f-47bd-b979-0fc81920a6a1,
 Type: None,
 Status: Completed), Run(Experiment: experiment-with-mlflow,
 Id: f4a3711b-ef77-46a6-ae49-142978c07cc1,
 Type: None,
 Status: Completed), Run(Experiment: experiment-with-mlflow,
 Id: 153632bc-3623-4808-bf8c-7c1fef1d17e5,
 Type: None,
 Status: Completed), Run(Experiment: experiment-with-mlflow,
 Id: experiment-with-mlflow_1561127517_8b46e0cb,
 Type: azureml.scriptrun,
 Status: Completed), Run(Experiment: experiment-with-mlflow,
 Id: experiment-with-mlflow_1561127667_c0060f62,
 Type: azureml.scriptrun,
 Status: Completed), Run(Experiment: experimen

Then, let's select the most recent training run and find its ID. You also need to specify the path in run history where the model was saved. 

In [4]:
runid = runs[0].id
model_save_path = "model"

### Create Docker image

To create a Docker image with Azure Machine Learning for Model Management, use ```mlflow.azureml.build_image``` method. Specify the model path, your workspace, run ID and other parameters.

MLflow automatically recognizes the model framework as scikit-learn, and creates the scoring logic and includes library dependencies for you.

Note that the image creation can take several minutes.

In [5]:
import mlflow.azureml

azure_image, azure_model = mlflow.azureml.build_image(model_uri="runs:/{}/{}".format(runid, model_save_path),
                                                      workspace=ws,
                                                      model_name='diabetes-sklearn-model',
                                                      image_name='diabetes-sklearn-image',
                                                      synchronous=True)

2020/01/08 14:26:10 INFO mlflow.azureml: Registered an Azure Model with name: `diabetes-sklearn-model` and version: `9`


Registering model diabetes-sklearn-model
Creating image


2020/01/08 14:26:30 INFO mlflow.azureml: Building an Azure Container Image with name: `diabetes-sklearn-image` and version: `8`


Running................................................................................
Succeeded
Image creation operation finished for image diabetes-sklearn-image:8, operation "Succeeded"


### Deploy web service

Let's use Azure Machine Learning SDK to deploy the image as a web service. 

First, specify the deployment configuration. Azure Container Instance is a suitable choice for a quick dev-test deployment, while Azure Kubernetes Service is suitable for scalable production deployments.

Then, deploy the image using Azure Machine Learning SDK's ```deploy_from_image``` method.

Note that the deployment can take several minutes.

In [7]:
%%time

from azureml.core.webservice import AciWebservice, Webservice


aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=1, 
                                                tags={"method" : "sklearn"}, 
                                                description='Diabetes model',
                                                location='eastus2')


# Deploy the image to Azure Container Instances (ACI) for real-time serving
webservice = Webservice.deploy_from_image(
    image=azure_image, workspace=ws, name="diabetes-model-2", deployment_config=aci_config)


webservice.wait_for_deployment(show_output=True)

Running...............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 192 ms, sys: 17.3 ms, total: 209 ms
Wall time: 2min 48s


## Make a scoring request

Let's take the first few rows of test data and score them using the web service

In [8]:
test_rows = [
    [0.01991321,  0.05068012,  0.10480869,  0.07007254, -0.03596778,
     -0.0266789 , -0.02499266, -0.00259226,  0.00371174,  0.04034337],
    [-0.01277963, -0.04464164,  0.06061839,  0.05285819,  0.04796534,
     0.02937467, -0.01762938,  0.03430886,  0.0702113 ,  0.00720652],
    [ 0.03807591,  0.05068012,  0.00888341,  0.04252958, -0.04284755,
     -0.02104223, -0.03971921, -0.00259226, -0.01811827,  0.00720652]]

MLflow-based web service for scikit-learn model requires the data to be converted to Pandas DataFrame, and then serialized as JSON. 

In [9]:
import json
import pandas as pd

test_rows_as_json = pd.DataFrame(test_rows).to_json(orient="split")

Let's pass the conveted and serialized data to web service to get the predictions.

In [10]:
predictions = webservice.run(test_rows_as_json)

print(predictions)

[232.5831070160765, 243.5417219310129, 163.65557038346532]


You can use the web service's scoring URI to make a raw HTTP request

In [11]:
webservice.scoring_uri

'http://d200e405-2945-4109-8f65-482c75d3a6f7.eastus2.azurecontainer.io/score'

You can diagnose the web service using ```get_logs``` method.

In [12]:
webservice.get_logs()

'2020-01-08T14:36:20,583927730+00:00 - rsyslog/run \n2020-01-08T14:36:20,585475929+00:00 - nginx/run \n2020-01-08T14:36:20,584440130+00:00 - gunicorn/run \n2020-01-08T14:36:20,585547829+00:00 - iot-server/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2020-01-08T14:36:20,833641288+00:00 - iot-server/finish 1 0\n2020-01-08T14:36:20,906636076+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 19.6.0\nListening at: http://127.0.0.1:31311 (10)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 47\nInitializing logger\nStarting up app insights client\nStarting up request id generator\nStarting up app insight hooks\nInvoking user\'s init function\n2020-01-08 14:36:25,925 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:\n\tMessage: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.\n\tInnerExcepti